# The capital asset pricing model and the security market line

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
"""
Linear regression with SciPy
"""
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)

In [ ]:
print(beta, alpha)

0.5077431878770808 -0.008481900352462384


# Multivariate linear regression of factor models

In [ ]:
"""
Least squares regression with statsmodels
"""
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                 -0.327
Method:                 Least Squares   F-statistic:                    0.7184
Date:                Tue, 29 Jul 2025   Prob (F-statistic):              0.723
Time:                        18:39:14   Log-Likelihood:                 4.4368
No. Observations:                   9   AIC:                             7.126
Df Residuals:                       1   BIC:                             8.704
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4212      8.115      0.052      0.9

In [ ]:
print(results.params)

[ 0.42124651  0.19645334 -1.30424896  0.37583113 -0.57744251 -0.3590552
  0.42172085  1.50374351]


# Linear optimization

## A maximization example with linear programming

In [ ]:
# FIX: Install the pulp library for linear optimization
!pip install pulp

import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 37.9 MB/s eta 0:00:00


In [ ]:
"""
A linear optimization problem with 2 variables
"""
import pulp

x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)

problem = pulp.LpProblem(
    'A simple maximization objective',
    pulp.LpMaximize)
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem.solve()

1

In [ ]:
print("Maximization Results:")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


## A minimization example with integer programming

In [ ]:
"""
An example of implementing an integer
programming model with binary conditions
"""
import pulp

dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts('quantity',
                                   dealers,
                                   lowBound=0,
                                   cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts('orders',
                                  dealers,
                                  cat=pulp.LpBinary)

In [ ]:
import pulp

# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)

# FIX: The objective function is now linear.
model += sum([
    (variable_costs[i] * quantities[i] + fixed_costs[i] * is_orders[i])
    for i in dealers
]), 'Minimize portfolio cost'

# --- The rest of the constraints are added below ---

# Total contracts required
model += sum([quantities[i] for i in dealers]) == 150, 'Total contracts required'

# Boundary constraints for each dealer
model += 30 <= quantities['X'] <= 100, 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90, 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70, 'Boundary of total volume of Z'

# FIX: Add constraints to link the 'is_orders' binary variable to the 'quantities'
# This ensures that if any quantity is ordered from a dealer, the fixed cost is applied.
for i in dealers:
    model += quantities[i] <= 1000 * is_orders[i] # The "big M" method

# Solve the (now correct) model
model.solve()

# Print the results
print("Optimal solution:")
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

Optimal solution:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0


In [ ]:
"""
This is an example of implementing an
IP model with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X'
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()

1

In [ ]:
print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)

print('Total cost:',  pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


# Solving linear equations using matrices

In [ ]:
"""
Linear algebra with NumPy matrices
"""
import numpy as np

A = np.array([[2, 1, 1],[1, 3, 2],[1, 0, 0]])
B = np.array([4, 5, 6])

Use the `linalg.solve` function of NumPy to solve a system of linear scalar
equations:

In [ ]:
print(np.linalg.solve(A, B))

[  6.  15. -23.]


# The LU decomposition

In [ ]:
"""
LU decomposition with SciPy
"""
import numpy as np
import scipy.linalg as linalg

# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]
])
B = np.array([4., 5., 6.])

# Perform

In [ ]:
print(x)

x


In [ ]:
import scipy

P, L, U = scipy.linalg.lu(A)

print('P=\n', P)
print('L=\n', L)
print('U=\n', U)

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


# The Cholesky decomposition

In [ ]:
"""
Cholesky decomposition with NumPy
"""
import numpy as np

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

In [ ]:
print(L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [ ]:
print(np.dot(L, L.T.conj()))  # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [ ]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [ ]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y

In [ ]:
print(x)

[ 1.  2. -1.  1.]


In [ ]:
# FIX: Use the modern '@' operator for matrix multiplication instead of np.mat
# This calculates B = Ax to verify the solution.
print(A @ x)

[  6.  25. -11.  15.]


# The QR decomposition

In [ ]:
"""
QR decomposition with scipy
"""
import numpy as np
import scipy.linalg as linalg


A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])

Q, R = scipy.linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = scipy.linalg.solve(R, y)  # Solve Rx=y

In [ ]:
print(x)

[  6.  15. -23.]


# Solving with other matrix algebra methods

## The Jacobi method

In [ ]:
"""
Solve Ax=B with the Jacobi method
"""
import numpy as np

def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [ ]:
A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [ ]:
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


# The Gauss-Seidel method

In [ ]:
"""
Solve Ax=B with the Gauss-Seidel method
"""
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A-L  # decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)

    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [ ]:
A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

In [ ]:
print('x', '=', x)

x = [ 1.  2. -1.  1.]
